In [1]:
import transformers
from datasets import load_dataset, load_metric, load_from_disk, Dataset, DatasetDict
from evaluate import load
import numpy as np
import pandas as pd
import torch
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\danii\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [6]:
data = pd.read_csv('train.csv')
metric = load('rouge')
model_checkpoints = 'facebook/bart-large-xsum'
tokenizer = transformers.AutoTokenizer.from_pretrained(model_checkpoints)

In [9]:
len(tokenizer.encode(data.abstract.max(), return_tensors='pt')[0])

186

In [10]:
len(tokenizer.encode(data.title.max(), return_tensors='pt')[0])

19

In [43]:
train = Dataset.from_pandas(data[:120000])
val = Dataset.from_pandas(data[125000:])
test = Dataset.from_pandas(data[130000:])

data = DatasetDict({'train':train, 'validation':val, 'test':test})

In [44]:
data

DatasetDict({
    train: Dataset({
        features: ['abstract', 'title'],
        num_rows: 120000
    })
    validation: Dataset({
        features: ['abstract', 'title'],
        num_rows: 10000
    })
    test: Dataset({
        features: ['abstract', 'title'],
        num_rows: 5000
    })
})

In [11]:
max_input = 256
max_target = 32

In [52]:
def preprocess_data(data_to_process):
    inputs = [text for text in data_to_process['abstract']]
    model_inputs = tokenizer(inputs, max_length=max_input, padding='max_length', truncation=True)

    with tokenizer.as_target_tokenizer():
        targets = tokenizer(data_to_process['title'], max_length=max_target, padding='max_length', truncation=True)
    
    model_inputs['labels'] = targets['input_ids']
    return model_inputs

In [53]:
tokenized_data = data.map(preprocess_data, batched=True, remove_columns=['abstract', 'title'])

  0%|          | 0/120 [00:00<?, ?ba/s]

c:\Users\danii\PycharmProjects\pythonProject\venv\lib\site-packages\transformers\tokenization_utils_base.py:3546: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


  0%|          | 0/10 [00:00<?, ?ba/s]

  0%|          | 0/5 [00:00<?, ?ba/s]

In [54]:
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(model_checkpoints)

Downloading:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

c:\Users\danii\PycharmProjects\pythonProject\venv\lib\site-packages\huggingface_hub\file_download.py:123: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\danii\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [55]:
collator = transformers.DataCollatorForSeq2Seq(tokenizer, model=model)

In [56]:
batch_size = 8

In [57]:
def compute_rouge(pred):
    predictions, labels = pred
    decode_predictions = tokenizer.batch_decode(predictions, skip_special_tokens=True)
    decode_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    res = metric.compute(predictions=decode_predictions, references=decode_labels, use_stemmer=True)
    res = {key: value*100 for key, value in res.items()}

    pred_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
    res['gen_len'] = np.mean(pred_lens)

    return {k: round(v, 4) for k,v in res.items()}

In [58]:
args = transformers.Seq2SeqTrainingArguments(
    'conversation-summ',
    evaluation_strategy='epoch',
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    gradient_accumulation_steps=2,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=1,
    predict_with_generate=True,
    eval_accumulation_steps=1
)

In [59]:
trainer = transformers.Seq2SeqTrainer(
    model,
    args,
    train_dataset=tokenized_data['train'],
    eval_dataset=tokenized_data['validation'],
    data_collator=collator,
    tokenizer=tokenizer,
    compute_metrics=compute_rouge
)

In [ ]:
trainer.train()

In [ ]:
trainer.save_model("./my_model")

In [2]:
test_data = pd.read_csv('test.csv')

In [4]:
texts = test_data.abstract.values

In [ ]:
for i in range(len(texts)):
    model_inputs = tokenizer(texts[i], max_length=max_input, padding='max_length', truncation=True)
    pred, _, _ = trainer.predict([model_inputs])
    texts[i] = tokenizer.decode(pred[0], skip_special_tokens=True)

In [ ]:
path = 'path_to_my_model'
model = transformers.AutoModelForSeq2SeqLM.from_pretrained(path)
tokenizer = transformers.AutoTokenizer.from_pretrained(path)